In [1]:
import numpy as np
import xarray as xr
from datetime import datetime as dt
import sys
import matplotlib.pyplot as plt
import cm_xml_to_matplotlib as cm # Note: this python script must be located in the same folder as the current script
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import os
import xesmf as xe

In [2]:
## Import data

## CAM Hindcast data will be imported into a list. Each index represents the number of hindcast day

# Setup strings to be used to create the path to each file
hcastDayStr = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
dri = '/lss/research/agon-lab/CAM_hindcasts/tratl_cam5.3_ne30_rneale.globe/'
fn = 'tratl_cam5.3_ne30_rneale.globe.fcast.day' 

# Initialize empty list that will contain all the specified hindcast days
dataPRECTRaw = [None] * (len(hcastDayStr)+1)
dataLH3hr  = [None] * (len(hcastDayStr)+1)
dataSH3hr  = [None] * (len(hcastDayStr)+1)

# Iterate through the string that dictates which hindcast days will be loaded
for ii,day in enumerate(hcastDayStr):
#     print(dri+fn+day+'.PRECT.nc')
    dataPRECTRaw[ii+1] = xr.open_dataset(dri+fn+day+'.PRECT.nc')
    dataLH3hr[ii+1]  = xr.open_dataset(dri+fn+day+'.LHFLX.nc')
    dataSH3hr[ii+1]  = xr.open_dataset(dri+fn+day+'.SHFLX.nc')
    
## Observation precipitation

driPRECTObs = '/lss/research/agon-lab/TMPA-daily/' # Used for Hadley computer
# driPRECTObs = '/home/jlarson1/data/' # Used for agron computer
fnPRECTObs  = 'pr_daily_2009-2010.nc'
dataPRECTObs = xr.open_dataset(driPRECTObs + fnPRECTObs)

# ## Observation surface winds

# # Note that these datasets were created specifically for this set of hindcasts. 
# # For future reference, if a more generic dataset(s) is used, the code in the next few sections might be useful
# # driWindObs = '/home/jlarson1/data/ERA5/' # Used for agron
# driWindObs = '/home/jlarson1/data/' # Used for Hadley
# fnUBOTObs = 'u10m_2009-2010-partial_daily.nc' ; fnVBOTObs = 'v10m_2009-2010-partial_daily.nc'
# dataUBOTObs = xr.open_dataset(driWindObs + fnUBOTObs)
# dataVBOTObs = xr.open_dataset(driWindObs + fnVBOTObs)

In [3]:
# Convert time stamps from cftime to datetime format

# Initialize array to hold every day's (152 days) time stamp for each hindcast day (20 days)
prectTime = np.array([ [None] * len(dataPRECTRaw[1]['time']) ] * len(dataPRECTRaw))

# Loop through all hindcast days
# Start the index at 1 so the indexing is more intuitive (e.g. day 7 hindcast is indexed with a 7)
for ii in range(1,  len(dataPRECTRaw) ):
    # Loop through each day in the hindcast dataset
    for jj in range( len(dataPRECTRaw[1]['time']) ):
        # Parse through the time stamp and convert to the datetime class
        prectTime[ii][jj] = dt.strptime(str(dataPRECTRaw[ii]['time'][jj].values), '%Y-%m-%d %H:%M:%S')

In [ ]:
# Time average the 3-hourly wind hindcast data to get daily wind data

# Create an empty array to hold the daily averaged data
lh = np.zeros([len(hcastDayStr)+1, len(dataLH3hr[1]['time'])//8, len(dataLH3hr[1]['lat']), len(dataLH3hr[1]['lon'])])
sh = np.zeros([len(hcastDayStr)+1, len(dataSH3hr[1]['time'])//8, len(dataSH3hr[1]['lat']), len(dataSH3hr[1]['lon'])])

# Iterate for the number of hindcast days
for ii in range(1, len(lh) ):

    # Iterate for the number of time stamps in 3hrly data. Count by 8
    for jj in range(0, len(dataLH3hr[1]['LHFLX']), 8):
        # On the left, jj is divided by 8 so it increases by 1 (each day)
        # On the right, 8 is added to jj to include all 8 daily data points in the mean
        lh[ii][jj//8] = dataLH3hr[ii]['LHFLX'][jj:jj+8].mean(dim='time').values
        sh[ii][jj//8] = dataSH3hr[ii]['SHFLX'][jj:jj+8].mean(dim='time').values
        

In [ ]:
# Organize the daily horizontal wind data into a dataset
# Also, recreate the precip datasets with the new datetime-formatted time stamps

# Initialize empty arrays to hold the new dataArrays
dataLH  = [None] * (len(hcastDayStr)+1)
dataSH  = [None] * (len(hcastDayStr)+1)
dataPRECT = [None] * (len(hcastDayStr)+1)

# Loop through list and create dataArrays
for ii in range(1, len(lh) ):
    dataLH[ii]  = xr.DataArray(lh[ii], dims=['time','lat','lon'], 
                                coords={'time':prectTime[ii], 'lat':dataLH3hr[1]['lat'], 'lon':dataLH3hr[1]['lon']})
    
    dataSH[ii]  = xr.DataArray(sh[ii], dims=['time','lat','lon'],
                                coords={'time':prectTime[ii], 'lat':dataSH3hr[1]['lat'], 'lon':dataSH3hr[1]['lon']})
    
    # Convert precip from m/s to mm/day
    dataPRECT[ii] = xr.DataArray(dataPRECTRaw[ii]['PRECT']*1000*86400, dims=['time','lat','lon'],
                                coords={'time':prectTime[ii], 'lat':dataPRECTRaw[1]['lat'], 'lon':dataPRECTRaw[1]['lon']})

In [ ]:
# ************************************************************************************************
# *************************** USER INPUTS ARE REQUIRED IN THIS CELL ******************************
# ************************************************************************************************


#### Select the date in 'YYYY-MM-DD' format (ISO 8601) ####
_dayOfTheSeason = '2010-02-23' 

#### Select the number of days to be time averaged ####
numDayAvg = 6

# Choose region of interest 
_lats =  -20
_latn =  20
lonw = -135 # *** For longitudes west of the Prime Meridian, enter a negative number ***
lone = -90  

# User inputs no longer required 

# ************************************************************************************************
# ************************************************************************************************
# ************************************************************************************************

# Convert _dayOfTheSeason to datetime64 format
dayOfTheSeason = np.datetime64(_dayOfTheSeason)

# Increase latitude range for interpolation purposes
lats = _lats-0.5
latn = _latn+0.5

In [ ]:
## Select the data within the user defined region of interest

# First, check if the user has requested data that is out of the time bounds of hindcast dataset
# If so, null all relevant variables to prevent any erroneous plots
if dayOfTheSeason+numDayAvg-1 > dataPRECT[1]['time'][-1] or dayOfTheSeason < dataPRECT[-1]['time'][0]:
#     prectSel=None; vbotSel=None; ubotSel=None; vbotObsSel=None; ubotObsSel=None; prectObsSel=None
#     prectPlot=None; vbotPlot=None; ubotPlot=None; VbotObsPlot=None; ubotObsPlot=None; prectObsPlot=None
    sys.exit("The user has attemped to access data that is outside of the time bounds of the hindcast dataset")

# Select the data within the region of interest
prectSel = [None] * len(dataPRECT)
lhSel  = [None] * len(dataLH)
shSel  = [None] * len(dataSH)

# Iterate through each hindcast day to select the data in the region of interest
# The iteration starts at 1 to agree with the indexing convention of the hindcast days (starting with 1)
for ii in range(1,len(dataPRECT)): 
    # Subtract 1 from numDayAvg since a slice will include the end point
    # Add 360 to the longitude so that it agrees with the dataset which ranges from 0 to 360
    prectSel[ii] = dataPRECT[ii].sel(time=slice(dayOfTheSeason,dayOfTheSeason+numDayAvg-1), lat=slice(lats,latn), lon=slice(lonw+360,lone+360))
    lhSel[ ii] = dataLH[ ii].sel(time=slice(dayOfTheSeason,dayOfTheSeason+numDayAvg-1), lat=slice(lats,latn), lon=slice(lonw+360,lone+360))
    shSel[ ii] = dataSH[ ii].sel(time=slice(dayOfTheSeason,dayOfTheSeason+numDayAvg-1), lat=slice(lats,latn), lon=slice(lonw+360,lone+360))

# prectObsSel = dataPRECTObs['pr'].sel(time=slice(dayOfTheSeason, dayOfTheSeason+numDayAvg-1), lat=slice(lats,latn), lon=slice(lonw,lone))    

# # Note that the latitude slice is from north to south to match the wind obs convention. 
# # Also, numDayAvg does not need to be subtracted by 1. Presumably because the obs data has a time stamp on it
# vbotObsSel = dataVBOTObs['v10'].sel(time=slice(dayOfTheSeason, dayOfTheSeason+numDayAvg), 
#                              latitude=slice(latn,lats), longitude=slice(lonw+360,lone+360))

# ubotObsSel = dataUBOTObs['u10'].sel(time=slice(dayOfTheSeason, dayOfTheSeason+numDayAvg), 
#                              latitude=slice(latn,lats), longitude=slice(lonw+360,lone+360))

In [ ]:
# Average the data for the length of the numDayAvg specificed 

# Initialize lists for hindcast data
prectPlot = [None] * len(prectSel)
lhPlot  = [None] * len(lhSel)
shPlot  = [None] * len(shSel)

# Perform time average for all hindcast days
for ii in range(1, len(prectSel)):
    prectPlot[ii] = prectSel[ii].mean(dim='time')
    lhPlot[ ii] = lhSel[ ii].mean(dim='time')
    shPlot[ ii] = shSel[ ii].mean(dim='time')

# Perform average for obs data
# vbotObsPlot  = vbotObsSel.mean( dim='time')
# ubotObsPlot  = ubotObsSel.mean( dim='time')
# prectObsPlot = prectObsSel.mean(dim='time')

In [ ]:
# Plot four subplots for each hindcast day

##### Control plotting variables here #####

# Set which days are plotted
hcastDay = [5,10,15,20]
# hcastDay = [1,2,3,4]

# Set the max value of the colorbar
# Vmax = 30

# Set the density of the wind vector arrows. A greater integer reduces the density
# n = int(3)

###########################################

# Set border of precip data
image_extent = [lonw,lone,lats,latn]

# Pre-generate the subplots with a PlateCarree projection
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(11, 8.5), subplot_kw=dict(projection=ccrs.PlateCarree()))

# Prepare colormap
# mycmap = cm.make_cmap('CustomColorMap1.xml')
mycmap = cm.make_cmap('LarsonColormap.xml')

# Iterate through the subplots
for ii, day in enumerate(hcastDay):
    
    # Plot precip in mm/day
    im = axes.flat[ii].imshow(lhPlot[day], extent=image_extent, origin='lower', # vmax=Vmax
                              interpolation='bilinear', cmap=plt.get_cmap(mycmap))

    # Plot horizontal wind vectors. The density of these vectors is controlled by n
#     quiv = axes.flat[ii].quiver(vbotPlot[day]['lon'][::n],   vbotPlot[day]['lat'][::n], ubotPlot[day][::n,::n], 
#                                 vbotPlot[day][::n,::n], transform=ccrs.PlateCarree(), width=0.0065)#, headlength=3, headwidth=2)
    
    # Set boundaries of graphs
    axes.flat[ii].set_xlim([lonw,lone])
    axes.flat[ii].set_ylim([_lats,_latn]) # Use user specified lat values
    
    # Set asthetic constraints of the plot
    axes.flat[ii].set_xticks(np.arange(lonw,lone+1,15), crs=ccrs.PlateCarree())
    axes.flat[ii].set_yticks(np.arange(_lats,_latn+1,5),  crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    axes.flat[ii].xaxis.set_major_formatter(lon_formatter)
    axes.flat[ii].yaxis.set_major_formatter(lat_formatter)
    axes.flat[ii].coastlines(resolution='110m')
    axes.flat[ii].gridlines(xlocs=np.arange(lonw,lone+1,15), ylocs=np.arange(_lats,_latn+1,5)) 
    axes.flat[ii].title.set_text('day '+str(day)+' hindcast')
    axes.flat[ii].axhline(0, color='0', linewidth = 1.5)
    
# Create reference vector for quiver
# qk = axes.flat[ii].quiverkey(quiv, 0.84, 2.251, 8, r'8 [m sec$^{-1}$]', labelpos='E', fontproperties={'size':'large'})
    
# The following code creates a new axes for the colorbar
fig.subplots_adjust(right=0.825) #0.825
# Add base axis for color bar. Located at 85% from left and 15% from bottom. Width is 2% of figure. Height is 70% of figure.
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7]) 
fig.colorbar(im, cax=cbar_ax, extend='both')

# Set title for the entire figure
fig.suptitle("Hindcast Latent Heat Flux for "+str(dayOfTheSeason)+" and a "+str(numDayAvg)+" day average\n",
             size=16,y=0.95)

## Save figure code    
# plt.savefig(dir+'/'+str(dayOfTheSeason)+'with'+str(numDayAvg)+'DayAvgHindcastMapPlot', dpi=_dpi, bbox_inches='tight')

plt.show()

In [ ]:
# Plot four subplots for each hindcast day

##### Control plotting variables here #####

# Set which days are plotted
hcastDay = [5,10,15,20]
# hcastDay = [1,2,3,4]

# Set the max value of the colorbar
# Vmax = 30

# Set the density of the wind vector arrows. A greater integer reduces the density
# n = int(3)

###########################################

# Set border of precip data
image_extent = [lonw,lone,lats,latn]

# Pre-generate the subplots with a PlateCarree projection
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(11, 8.5), subplot_kw=dict(projection=ccrs.PlateCarree()))

# Prepare colormap
# mycmap = cm.make_cmap('CustomColorMap1.xml')
mycmap = cm.make_cmap('LarsonColormap.xml')

# Iterate through the subplots
for ii, day in enumerate(hcastDay):
    
    # Plot precip in mm/day
    im = axes.flat[ii].imshow(shPlot[day], extent=image_extent, origin='lower', # vmax=Vmax
                              interpolation='bilinear', cmap=plt.get_cmap(mycmap))

    # Plot horizontal wind vectors. The density of these vectors is controlled by n
#     quiv = axes.flat[ii].quiver(vbotPlot[day]['lon'][::n],   vbotPlot[day]['lat'][::n], ubotPlot[day][::n,::n], 
#                                 vbotPlot[day][::n,::n], transform=ccrs.PlateCarree(), width=0.0065)#, headlength=3, headwidth=2)
    
    # Set boundaries of graphs
    axes.flat[ii].set_xlim([lonw,lone])
    axes.flat[ii].set_ylim([_lats,_latn]) # Use user specified lat values
    
    # Set asthetic constraints of the plot
    axes.flat[ii].set_xticks(np.arange(lonw,lone+1,15), crs=ccrs.PlateCarree())
    axes.flat[ii].set_yticks(np.arange(_lats,_latn+1,5),  crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    axes.flat[ii].xaxis.set_major_formatter(lon_formatter)
    axes.flat[ii].yaxis.set_major_formatter(lat_formatter)
    axes.flat[ii].coastlines(resolution='110m')
    axes.flat[ii].gridlines(xlocs=np.arange(lonw,lone+1,15), ylocs=np.arange(_lats,_latn+1,5)) 
    axes.flat[ii].title.set_text('day '+str(day)+' hindcast')
    axes.flat[ii].axhline(0, color='0', linewidth = 1.5)
    
# Create reference vector for quiver
# qk = axes.flat[ii].quiverkey(quiv, 0.84, 2.251, 8, r'8 [m sec$^{-1}$]', labelpos='E', fontproperties={'size':'large'})
    
# The following code creates a new axes for the colorbar
fig.subplots_adjust(right=0.825) #0.825
# Add base axis for color bar. Located at 85% from left and 15% from bottom. Width is 2% of figure. Height is 70% of figure.
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7]) 
fig.colorbar(im, cax=cbar_ax, extend='both')

# Set title for the entire figure
fig.suptitle("Hindcast Sensible Heat Flux for "+str(dayOfTheSeason)+" and a "+str(numDayAvg)+" day average\n",
             size=16,y=0.95)

## Save figure code    
# plt.savefig(dir+'/'+str(dayOfTheSeason)+'with'+str(numDayAvg)+'DayAvgHindcastMapPlot', dpi=_dpi, bbox_inches='tight')

plt.show()